# Derived Table

This is a variant of the Subquery with the exception that, unlike the previous subquery examples we have seen, this is used within the FROM portion of your query.

What we are doing with a derived table is using the result of a query as a table to which we can then join.  Hence we are deriving a table and then using it in our query.

#### Example

Let's use an example in which we wish to see the following result:
* From hr.employee
 * Returning - employeeID, firstName, lastName, salary, dob, managerID
* Join sales.salesOrderHeader
 * Returning - each employee's total sales for 2018

We now know how to write this query using our JOINs and our knowledge of Aggregations, so let's just dive right in:

In [ ]:
select e.employeeID, e.firstName, e.lastName, e.salary, e.dob,
		e.managerID, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where s.orderDate between '2018-01-01' and '2018-12-31 23:59:59.999'
group by e.employeeID, e.firstName, e.lastName, e.salary,
			e.dob, e.managerID;

However, this can look quite complex with a long winded GROUP BY clause and messy syntax.

What if we could be a little more declarative?

'Return the sales for each employee for 2018 and JOIN this to the employee table'

Hence this seems like it should be a lot clearer to code.  Therefore let's do this a little more literally and see what happens.

Firstly we shall get the total sales per employee for 2018:

In [ ]:
select salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2018-01-01' and '2018-12-31 23:59:59.999'
group by salesPersonID;

Now we can get our hr.employee table and join our total Sales to these records:

In [ ]:
select e.employeeID, e.firstName, e.lastName, e.salary, e.dob,
		e.managerID, totalSales
from JupyterDatabase.hr.employee as e
join
(
	select salesPersonID, sum(totalDue) as totalSales
	from JupyterDatabase.sales.salesOrderHeader
	where orderDate between '2018-01-01' and '2018-12-31 23:59:59.999'
	group by salesPersonID
) as s
on e.employeeID = s.salesPersonID;

Although this may seem a longer query, it can be a lot clearer to understand its purpose.

#### Notes

Because a derived table is a table that we are then using in our wider query, a derived table must have column names.

We must also alias the derived table when referencing it in the same way we would a standalone table.

For example, this is NOT valid:

<code>SELECT *<br/>
FROM<br/>
(<br/>
&nbsp&nbsp&nbsp&nbspSELECT sum(column)<br/>
&nbsp&nbsp&nbsp&nbspFROM myTable<br/>
)</code>

We need to ensure that all columns in the derived table have aliases along with the resulting table itself.

This, therefore, is now correct:

<code>SELECT *<br/>
FROM<br/>
(<br/>
&nbsp&nbsp&nbsp&nbspSELECT sum(column) AS myTotal<br/>
&nbsp&nbsp&nbsp&nbspFROM myTable<br/>
) AS tableAlias</code>